In [83]:
import pandas as pd
import json
from tqdm import tqdm

In [10]:
import requests

# URL de la API
url_api = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'

# Datos de autenticación
clientID = "3035de74-9dd5-4223-93fb-2ba65bddc22f"
passkey = '57E940AAE9FDD0E192487B24CD0CE5C47B506F8DF8BFD8F1F71137F1DE9191F932D8974AE15530EA59AC853B6637695EC8FCA0B45CC99D7B123EBF957E62FD30'

headers = {
    'X-ClientId': clientID,
    'passKey': passkey
}

try:
    # Realizar la solicitud GET
    response = requests.get(url_api, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Procesar la respuesta JSON
        data = response.json()
        # Aquí puedes trabajar con los datos recibidos
    else:
        print("Error en la solicitud:", response.status_code)

except requests.exceptions.RequestException as e:
    # Capturar excepciones de solicitud
    print("Error en la solicitud:", e)

In [11]:
data

{'code': '01',
 'description': 'Token extend  into control-cache Data recovered  OK, (lapsed: 87 millsecs)',
 'datetime': '2024-03-02T19:23:49.849285',
 'data': [{'nameApp': 'Retrieve Stop info',
   'levelApp': 0,
   'updatedAt': '2024-02-13T14:15:12.150000',
   'userName': 'davicichu',
   'lastUpdate': {'$date': 1709403590401},
   'idUser': 'BF18F925-A1D6-4A46-9C43-696A4D87A8D8',
   'priv': 'U',
   'tokenSecExpiration': 86400,
   'email': 'david.lpmartin@gmail.com',
   'tokenDteExpiration': {'$date': 1709493590401},
   'flagAdvise': False,
   'accessToken': '754dac88-d8c1-11ee-82f2-26e8a1204cbd',
   'apiCounter': {'current': 1,
    'dailyUse': 250000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!'},
   'username': 'davicichu'}]}

### Obtener información a partir del código de parada

In [14]:
import requests

# URL de la API
url_api = 'https://openapi.emtmadrid.es/v1/transport/busemtmad/stops/88/detail/'

# Datos de autenticación
accessToken = '754dac88-d8c1-11ee-82f2-26e8a1204cbd'

headers = {
    'accessToken': accessToken
}

try:
    # Realizar la solicitud GET
    response = requests.get(url_api, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Procesar la respuesta JSON
        data = response.json()
        # Aquí puedes trabajar con los datos recibidos
    else:
        print("Error en la solicitud:", response.status_code)

except requests.exceptions.RequestException as e:
    # Capturar excepciones de solicitud
    print("Error en la solicitud:", e)

In [15]:
data

{'code': '90',
 'description': 'Error managing internal services',
 'datetime': '2024-03-02T19:23:58.055452',
 'data': []}

### Otro metodo para obtener la lat y lon

Realizar la peticion directamente a traves de la pagina si no da error, a esta url https://openapi.emtmadrid.es/v1/transport/busemtmad/stops/list/. Se obtiene un json de todas las paradas y su localización 

In [102]:
import json
import pandas as pd
with open('../data/emt_info.json', 'r', encoding='utf-8') as archivo:
    json_emt = json.load(archivo)

In [106]:
lista_paradas = json_emt.get('data')
latitud_list = []
longitud_list = []
codparada_list = []

for parada in lista_paradas:
    longitud_list.append(parada.get('geometry').get('coordinates')[0])
    latitud_list.append(parada.get('geometry').get('coordinates')[1])
    codparada_list.append(parada.get('node'))


In [170]:
coord_df = pd.DataFrame(
    {'codigo_parada':codparada_list, 'longitud':longitud_list, 'latitud':latitud_list}
)
coord_df

,codigo_parada,longitud,latitud
0,1,-3.782883,40.470144
1,2,-3.785959,40.468672
2,3,-3.783855,40.464957
3,4,-3.784097,40.461442
4,5,-3.784291,40.459436
...,...,...,...
4838,51030,-3.753276,40.411219
4839,51031,-3.674577,40.500636
4840,51032,-3.674161,40.500575
4841,51033,-3.717790,40.433854


In [172]:
coord_df.to_csv('../data/longitude_latitude.csv', index=False)

##### Comprobar cuantas coinciden y faltan del csv-clean

In [3]:
import pandas as pd

emt_csv = pd.read_csv('../data/emt-data-clean.csv')
coord_df = pd.read_csv('../data/longitude_latitude.csv')
paradas = list(emt_csv['codigo_parada'].unique())
coor_paradas =list(coord_df['codigo_parada'].astype(int))
paradas_faltan = []

In [4]:
for parada in paradas:
    if parada not in coor_paradas:
        paradas_faltan.append(parada)

#### Obtener intenerario de cada linea con distancias

In [21]:
emt_csv = pd.read_csv('../data/emt-data-clean.csv')
emt_csv

,codigo_estacion,codigo_parada,nombre_parada,linea,direccion_parada,cp_parada,distrito_parada
0,3061,2522,AV.PABLO NERUDA-MALGRAT MAR,54,Avda de Pablo Neruda 2,28038,13
1,3061,2522,AV.PABLO NERUDA-MALGRAT MAR,142,Avda de Pablo Neruda 2,28038,13
2,3061,2522,AV.PABLO NERUDA-MALGRAT MAR,144,Avda de Pablo Neruda 2,28038,13
3,3007,1748,CERRO DEL CASTAÑAR-MORALZARZAL,134,Calle del Cerro del Castañar 167,28034,8
4,3066,2530,ANDALUCES-SAN CLAUDIO,54,Calle de los Andaluces 9,28038,13
...,...,...,...,...,...,...,...
11099,6872,5438,SECUNDINO ZUAZO-JOSEFINA ALDECOA,BR1,Avda Provisional Parque de Valdebebas Diez 99,28055,16
11100,6873,5433,SECUNDINO ZUAZO-FINA DE CALDERÓN,BR1,Calle Provisional Parque de Valdebebas Once 44,28055,16
11101,6874,5434,SECUNDINO ZUAZO-FINA DE CALDERÓN,BR1,Avda Provisional Parque de Valdebebas Diez 62,28055,16
11102,6876,5974,MARÍA DE LAS MERCEDES DE BORBÓN-GTA.ANTONIO BATET,BR1,Calle Provisional Parque de Valdebebas Veinti...,28055,16


In [51]:
# Obtener valores únicos de la columna 'linea'
valores_unicos = emt_csv['linea'].unique()

# Convertir numpy.ndarray a lista
lista_lineas = valores_unicos.tolist()
len(lista_lineas)

223

In [52]:
import requests

itinerario_lineas = []

for linea in lista_lineas:

    # URL de la API
    url_api = f'https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/{linea}/route/'

    # Datos de autenticación
    accessToken = '754dac88-d8c1-11ee-82f2-26e8a1204cbd'

    headers = {
        'accessToken': accessToken
    }

    try:
        # Realizar la solicitud GET
        response = requests.get(url_api, headers=headers)

        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            # Procesar la respuesta JSON
            data = response.json()
            # Aquí puedes trabajar con los datos recibidos
            itinerario_lineas.append(data)
        else:
            print("Error en la solicitud:", response.status_code)

    except requests.exceptions.RequestException as e:
        # Capturar excepciones de solicitud
        print("Error en la solicitud:", e)

En una dirección (toA)

In [105]:
datos = []
for linea in tqdm(itinerario_lineas):
    num_linea = linea.get('data').get('line')
    dicc = linea.get('data').get('stops').get('toA')
    for feature in dicc['features']:
        stopNum = feature['properties']['stopNum']
        latitude =  feature['geometry']['coordinates'][0]
        longitude = feature['geometry']['coordinates'][1]
        distance = feature['properties']['distance']
        linea = num_linea
        datos.append({'linea':linea, 'stopNum': stopNum,  'latitude':latitude, 'longitude': longitude,'distance': distance})

df_stops_toA = pd.DataFrame(datos)

100%|██████████| 223/223 [00:00<00:00, 84263.95it/s]


In [106]:
df_stops_toA.to_csv('../data/stops_toA.csv', index=False)

En otra dirección (toB)

In [103]:
datos = []
for linea in tqdm(itinerario_lineas):
    num_linea = linea.get('data').get('line')
    dicc = linea.get('data').get('stops').get('toB')
    for feature in dicc['features']:
        stopNum = feature['properties']['stopNum']
        longitude = feature['geometry']['coordinates'][0]
        latitude =  feature['geometry']['coordinates'][1]
        distance = feature['properties']['distance']
        linea = num_linea
        datos.append({'linea':linea, 'stopNum': stopNum, 'longitude': longitude, 'latitude':latitude, 'distance': distance})

df_stops_toB = pd.DataFrame(datos)

100%|██████████| 223/223 [00:00<00:00, 46344.75it/s]


In [104]:
df_stops_toB.to_csv('../data/stops_toB.csv', index=False)